In [ ]:
import numpy as np
import torch.nn as nn
import torch
import torch.nn.functional as F
from torch.utils.data import DataLoader
from practical_b_1 import ImageDataset,SortedImageDataset,NormalDataset
from load_mnist import load_mnist
from torchvision import datasets
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt

In [ ]:
criterion = nn.CrossEntropyLoss()
batch_size = 64
c=10
EPOCHS = 5
input_size = 28*28
output_size=10
loss_function = nn.NLLLoss()

In [ ]:
class Network(nn.Module):
    def __init__(self,num_neurons,input_size,output_size):
        super().__init__()
        hidden1 = 50
        hidden2 = 20
        hidden3 = 10
        self.conv = nn.Sequential(
          nn.Linear(input_size,hidden1),
          nn.ReLU(),
          nn.Linear(hidden1,hidden2),
          nn.ReLU(),
          nn.Linear(hidden2,hidden3),
          nn.ReLU(),
          nn.Linear(hidden3, output_size)
        )
    
    def forward(self, x):
        return nn.Softmax(dim=0)(self.conv(x))

train the model

In [ ]:
def train(model, dataloader: DataLoader, optimizer):
    
    epoch_loss=0.0
    for batch, (images, labels) in enumerate(dataloader):
        optimizer.zero_grad()
        output = model(images.view(-1, input_size))
        output=output.float()
        loss = loss_function(output.squeeze(), labels.long())
        epoch_loss += loss.item()
        loss.backward()
        optimizer.step()
    epoch_loss = epoch_loss / len(dataloader)
    print('loss :{}'.format(loss))    


train loop

In [ ]:
def trainloop(dataloader, model, optimizer,EPOCHS):
    for epoch in range(EPOCHS):
        train(dataloader=dataloader, model=model, optimizer=optimizer)

Intialize model,optimizer,trainDataset,testDataset,traindataloader,testdataloader

In [ ]:
model = Network(num_neurons=(20,10,7),input_size=input_size,output_size=output_size)
optimizer = torch.optim.SGD(model.parameters(), lr=0.001)

In [ ]:
def test_model(test_loader:DataLoader,EPOCHS:int,model):
    model.eval()
    accuracies=[]
    for epoch in range(EPOCHS):                                             
        with torch.no_grad():
                correct = 0
                total = 0
                for images, labels in test_loader:
                    outputs = model(images.view(-1, input_size))
                    _, predicted = torch.max(outputs.data, 1)
                    total += labels.size(0)
                    correct += (predicted == labels).sum().item()

                    
                accuracies.append(  (100 * correct / total))
                print('Accuracy : %d %%' % (100 * correct / total))

    plt.plot(range(EPOCHS),accuracies,'b-o',label='Accuracy');
    plt.legend()
    plt.show()        
    return accuracies            

### 2.1
#### a
Train the classes separately, i.e. filter the dataset and train for five epochs
on all images depicting a 0, then on all images depicting a 1, and so on.

In [ ]:
def train_classes_separtely_task_a(batch_size):
    classes = range(c)
    for label in classes:
        model_b = Network(num_neurons=(20,10,7),input_size=input_size,output_size=output_size)
        trainDataset = ImageDataset(type='training', label=label)
        data_loader = DataLoader(trainDataset, batch_size=batch_size,shuffle=False, drop_last=False)
        trainloop(dataloader=data_loader, model=model,
                optimizer=optimizer, EPOCHS=5)
        testingDataset = ImageDataset(type='testing', label=label)
        test_loader = DataLoader(testingDataset, batch_size=batch_size,shuffle=True, drop_last=False)
        print('test label: {}'.format(label))
        test_model(model=model_b,EPOCHS=EPOCHS,test_loader=test_loader)          

### 2.1
#### b
Sort the images by class and train them in order for 25 epochs. Make sure
that the DataLoader is not shuffling the data.

In [ ]:
def train_sorted_without_shuffule_task_b(batch_size):
    sortedDataSet=SortedImageDataset('training')
    sortedDataSetLoader = DataLoader(sortedDataSet, batch_size=batch_size,
                                                shuffle=False, drop_last=False)
    model_b = Network(num_neurons=(20,10,7),input_size=input_size,output_size=output_size)
    sortedEpoch=25
    trainloop(dataloader=sortedDataSetLoader, model=model_b,
                optimizer=optimizer, EPOCHS=sortedEpoch)    

    ggg=SortedImageDataset(type='testing')
    test_loader = DataLoader(ggg, batch_size=batch_size,shuffle=True, drop_last=False)
    test_model(model=model_b,EPOCHS=sortedEpoch,test_loader=test_loader)
                                         

### 2.1
#### c
Shuffle the images every epoch and train them for 25 epochs

In [ ]:
def train_with_shuffule_task_c(batch_size):
    normalData=NormalDataset('training')

    normalDataSetLoader = DataLoader(normalData, batch_size=batch_size,
                                                shuffle=True, drop_last=False)
    model_c = Network(num_neurons=(20,10,7),input_size=input_size,output_size=output_size)

    sortedEpoch=25

    trainloop(dataloader=normalDataSetLoader, model=model_c,
                optimizer=optimizer, EPOCHS=sortedEpoch)

    ggg=NormalDataset(type='testing')

    test_loader = DataLoader(ggg, batch_size=batch_size,shuffle=True, drop_last=False)

    test_model(model=model_c,EPOCHS=sortedEpoch,test_loader=test_loader)

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=8d525717-702e-4285-81d5-cd5945dff109' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>